## 서로소 집합 (Disjoint Sets)
* 서로소 부분 집합들로 나누어진 원소들의 데이터를 처리하기 위한 자료구조

In [1]:
# 특정 원소가 속한 집합 찾기
def find_parent(parent, x):
    # 루트노드가 아닐 경우 루트노드 찾을 때까지 재귀 호출
    # 루트노드일 경우: parent가 자기 자신. 루트노드 아닐 경우: parent가 다른 노드.
    if parent[x] != x:
        return find_parent(parent, parent[x])
    return parent[x]


# 두 원소가 속한 집합 합치기
def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b


# 노드의 개수와 간선(union 연산)의 개수 입력
v, e = map(int, input().split())
parent = [0] * (v + 1)  # 부모 테이블 초기화

for i in range(1, v + 1):
    parent[i] = i

#union 연산을 각각 수행
for i in range(e):
    a, b = map(int, input().split())
    union_parent(parent, a, b)

#각 원소가 속한 집합 출력
print('각 원소가 속한 집합: ', end='')
for i in range(1, v + 1):
    print(find_parent(parent, i), end=' ')

print()

print('부모 테이블: ', end='')
for i in range(1, v + 1):
    print(parent[i], end=' ')

각 원소가 속한 집합: 1 1 1 1 5 5 
부모 테이블:1 1 2 1 5 5 

### 경로 압축 기법
Line 5~7의 경우 비효율적인 코드임 --> 매 계산마다 재귀호출
개선 방법? --> 경로 압축


In [2]:
# Line 5~7의 코드를 다음과 같이 바꾸면 된다.
# 경로 압축 기법의 구현

def find_parent(parent, x):
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

### 서로소 집합의 활용
- 무방향 그래프 내에서 사이클을 판별할 때 사용할 수 있다.
- -- 방향 그래프에서의 사이클: DFS
- 판별 방법: 각 간선의 루트노드를 확인. 만약 루트노드가 같다면 Cycle이 발생한 것이고, 다르다면 union 연산 수행.

#### 서로소 집합을 활용한 사이클 판별

In [11]:
def find_parent(parent, x):
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]


def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b


v, e = map(int, input().split())

# parent = [[i] for i in range(v + 1)]
parent = [0] * (v + 1)
for i in range(1, v + 1):
    parent[i] = i

cycle = False

for i in range(e):
    a, b = map(int, input().split())
    # 사이클이 발생한 경우 종료
    if find_parent(parent, a) == find_parent(parent, b):
        cycle = True
        break
    else:
        union_parent(parent, a, b)

if cycle:
    print('그래프 내에 사이클이 존재합니다.')
else:
    print('사이클이 없는 그래프입니다.')

그래프 내에 사이클이 존재합니다.


Line 18 관련
parent를 선언할 때 반복문을 안쓰고 싶어서 별 뻘짓을 다해봤는데 다 실패함.
[[i] for i in range(v+1)] -> 리스트가 되어버리기 때문에 불가
([i] for i in range(v+1)] -> 객체 생성자 취급이 되어버려서 안됨.

## 신장 트리
* 하나의 그래프가 있을 떄 모든 노드를 포함하면서 사이클이 존재하지 않는 부분 그래프.
### 대표적인 알고리즘
- 크루스칼 알고리즘(최소 신장 트리 알고리즘) -> 그리디

In [2]:
def find_parent(parent, x):
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]


def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a > b:
        parent[a] = b
    else:
        parent[b] = a


v, e = map(int, input().split())
parent = [0] * (v + 1)

edges = []
result = 0

for i in range(1, v + 1):
    parent[i] = i

for _ in range(e):
    a, b, cost = map(int, input().split())
    edges.append((cost, a, b))

edges.sort(key=lambda x: x[0])

for edge in edges:
    cost, a, b = edge
    if find_parent(parent, a) != find_parent(parent, b):
        union_parent(parent, a, b)
        result += cost

print(result)

159


## 위상 정렬(Topology Sort)
* 방향 그래프의 모든 노드를 방향성에 거스르지 않도록 순서대로 나열하는 것
* 어떤 작업에 대한 일 처리 순서등을 스케쥴링 하는 용도로 사용
### 위상 정렬의 수행 과정
1. 진입차수가 0인 노드를 큐에 넣는다
2. 큐가 빌 때까지 하나씩 원소를 꺼내며 다음의 작업을 수행한다.
    - 꺼낸 원소에서 출발하는 간선을 그래프에서 제거
    - 새롭게 진입차수가 0이 된 노드를 큐에 삽입.

위상 정렬은 주어진 그래프에 사이클이 존재한다면 모든 원소를 방문하기 전에 큐가 비어버림.
- 대부분은 사이클이 존재하지 않는다고 명시해줌

In [4]:
from collections import deque

# v=노드의 개수, e=간선의 개수
v, e = map(int, input().split())
enter_dim = [0] * (v + 1)  # 진입 차수 저장
graph = [[] for i in range(v + 1)]

for _ in range(e):
    s, e = map(int, input().split())  # 간선에 대한 정보 등록
    graph[s].append(e)  # 간선에 대한 정보를 인접 리스트에 등록.
    enter_dim[e] += 1  # 간선의 끝점의 진입차수를 1 증가시킴

def topology_sort():
    ans=[] # 정답이 저장될 리스트
    q=deque() # dequeue 선언

    # 진입차수가 0인 노드들을 큐에 삽입
    for i in range(1, v+1):
        if enter_dim[i]==0:
            q.append(i)

    while q:
        cur=q.popleft() # 큐에서 원소 1개 꺼내기
        ans.append(cur) # 위상 정렬 결과가 저장될 리스트에 현재 노드 저장

        # 현재 노드와 연결된 간선의 끝점에 있는 노드의 진입차수 1씩 감소시키기
        for i in graph[cur]:
            enter_dim[i]-=1
            # 만약 진입차수가 0이 되었다면 다음 방문 노드 대기열에 넣음.
            if enter_dim[i]==0:
                q.append(i)

    for i in ans:
        print(i,end=' ')
    print()

topology_sort()

1 2 5 3 6 4 7 
